In [4]:
import torch
import torch.nn as nn
import torchvision
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev)

# Load observations from the mnist dataset. The observations are divided into a training set and a test set
mnist_train = torchvision.datasets.MNIST('./data', train=True, download=True)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_train = torch.zeros((mnist_train.targets.shape[0], 10))  # Create output tensor
y_train[torch.arange(mnist_train.targets.shape[0]), mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.MNIST('./data', train=False, download=True)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_test = torch.zeros((mnist_test.targets.shape[0], 10))  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]), mnist_test.targets] = 1  # Populate output

# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)

In [5]:
class ConvolutionalNeuralNetworkModel(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetworkModel, self).__init__()

        self.logits = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), 
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), 
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 1024),
            nn.Flatten(),
            nn.Linear(1024, 10)).to(device)

    # Predictor
    def f(self, x):
        return torch.softmax(self.logits(x), dim=1)

    # Cross Entropy loss
    def loss(self, x, y):
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1)).to(device)

    # Accuracy
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float()).to(device)

In [6]:
model = ConvolutionalNeuralNetworkModel()

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), 0.001)
for epoch in range(20):
    for batch in range(len(x_train_batches)):
        model.loss(x_train_batches[batch].to(device), y_train_batches[batch].to(device)).backward()  # Compute loss gradients
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    print("accuracy = %s" % model.accuracy(x_test.to(device), y_test.to(device))) 

C:\Users\Beast\AppData\Roaming\Python\Python36\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


accuracy = tensor(0.9783, device='cuda:0')
accuracy = tensor(0.9825, device='cuda:0')
accuracy = tensor(0.9857, device='cuda:0')
accuracy = tensor(0.9876, device='cuda:0')
accuracy = tensor(0.9840, device='cuda:0')
accuracy = tensor(0.9886, device='cuda:0')
accuracy = tensor(0.9856, device='cuda:0')
accuracy = tensor(0.9815, device='cuda:0')
accuracy = tensor(0.9887, device='cuda:0')
accuracy = tensor(0.9901, device='cuda:0')
accuracy = tensor(0.9873, device='cuda:0')
accuracy = tensor(0.9868, device='cuda:0')
accuracy = tensor(0.9828, device='cuda:0')
accuracy = tensor(0.9877, device='cuda:0')
accuracy = tensor(0.9850, device='cuda:0')
accuracy = tensor(0.9879, device='cuda:0')
accuracy = tensor(0.9883, device='cuda:0')
accuracy = tensor(0.9885, device='cuda:0')
accuracy = tensor(0.9897, device='cuda:0')
accuracy = tensor(0.9886, device='cuda:0')
